#  TOPIC MODELING --- >> 20 News group data

In [1]:
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, lemmatize
from gensim.models import CoherenceModel
import nltk
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
from gensim.models import HdpModel

C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','thank','you'])

In [3]:
from sklearn.datasets import fetch_20newsgroups
df_train = fetch_20newsgroups(subset='train', shuffle = True)
df_test = fetch_20newsgroups(subset='test', shuffle = True)

In [4]:
list(df_train.target_names)
print(list(df_train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [10]:
df_train.data[1]

"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

In [11]:
df_train.target_names[1]

'comp.graphics'

In [6]:
#### lemmatization and preprocessing

data_processed=[]
for c in range(df_train.filenames.shape[0]):
    doc=df_train.data[c].split()
    doc_out=[]
    for x in doc:
        if x not in stop_words:
            lemma = lemmatize(x,allowed_tags=re.compile('(NN|VB|RB)'))
            if lemma:
                doc_out = doc_out + [lemma[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
    data_processed.append(doc_out)


C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\pattern\text\__init__.py:625: DeprecationWarning: generator '_read' raised StopIteration
  dict.update(self, (x.split(" ")[:2] for x in _read(self._path) if len(x.split(" ")) > 1))
C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\pattern\text\__init__.py:959: DeprecationWarning: generator '_read' raised StopIteration
  for x in _read(self.path):
C:\Users\SAI KRISHNA\Anaconda3\lib\site-packages\pattern\text\__init__.py:2127: DeprecationWarning: generator '_read' raised StopIteration
  for v in _read(self._path):


In [12]:
print(" ".join(data_processed[1]))

guykuo guy kuo subject si clock poll call summary call si clock report keyword si article shelley organization university washington line nntp carson number brave soul upgrade si clock oscillator share experience poll please send message detail experience procedure top speed attain cpu rate speed add card adapter heat sink hour usage day disk functionality floppy especially request summarize next day please add network knowledge base do clock answer poll thank guy kuo guykuo


In [14]:
#### Creating a dictionary for term document frequency
id2word = corpora.Dictionary(data_processed)

In [15]:
id2word.filter_extremes(no_below=10, no_above=0.1, keep_n= 100000)

In [16]:
len(id2word)

8730

In [17]:
bow_corpus = [id2word.doc2bow(doc) for doc in data_processed]

In [18]:
len(bow_corpus)

11314

In [19]:
document_num = 1
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], id2word[bow_doc_x[i][0]],bow_doc_x[i][1]))

Word 24 ("adapter") appears 1 time.
Word 25 ("add") appears 2 time.
Word 26 ("answer") appears 1 time.
Word 27 ("attain") appears 1 time.
Word 28 ("base") appears 1 time.
Word 29 ("brave") appears 1 time.
Word 30 ("card") appears 1 time.
Word 31 ("carson") appears 1 time.
Word 32 ("clock") appears 4 time.
Word 33 ("cpu") appears 1 time.
Word 34 ("detail") appears 1 time.
Word 35 ("disk") appears 1 time.
Word 36 ("especially") appears 1 time.
Word 37 ("experience") appears 2 time.
Word 38 ("floppy") appears 1 time.
Word 39 ("functionality") appears 1 time.
Word 40 ("guy") appears 2 time.
Word 41 ("heat") appears 1 time.
Word 42 ("hour") appears 1 time.
Word 43 ("keyword") appears 1 time.
Word 44 ("knowledge") appears 1 time.
Word 45 ("kuo") appears 2 time.
Word 46 ("message") appears 1 time.
Word 47 ("network") appears 1 time.
Word 48 ("next") appears 1 time.
Word 49 ("oscillator") appears 1 time.
Word 50 ("poll") appears 3 time.
Word 51 ("procedure") appears 1 time.
Word 52 ("rate") ap

In [20]:
### Model---- LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,id2word=id2word,num_topics=7,random_state=100,update_every=1,chunksize=100,passes=10,alpha='auto',per_word_topics=True)


In [21]:
for num, topic in lda_model.print_topics(-1):
    print('Topic: {}, \n Words: {} \n'.format(num,topic))

Topic: 0, 
 Words: 0.010*"space" + 0.009*"chip" + 0.008*"information" + 0.008*"technology" + 0.007*"research" + 0.005*"group" + 0.005*"phone" + 0.005*"datum" + 0.005*"program" + 0.005*"science" 

Topic: 1, 
 Words: 0.026*"max" + 0.025*"car" + 0.012*"video" + 0.011*"color" + 0.011*"bike" + 0.010*"drive" + 0.010*"dn" + 0.009*"test" + 0.008*"sphere" + 0.007*"engine" 

Topic: 2, 
 Words: 0.016*"window" + 0.015*"file" + 0.014*"drive" + 0.014*"card" + 0.008*"program" + 0.008*"software" + 0.008*"driver" + 0.008*"mac" + 0.008*"sale" + 0.008*"machine" 

Topic: 3, 
 Words: 0.020*"team" + 0.019*"game" + 0.014*"play" + 0.010*"internet" + 0.010*"st" + 0.009*"player" + 0.009*"hockey" + 0.008*"vs" + 0.007*"bob" + 0.006*"season" 

Topic: 4, 
 Words: 0.006*"ll" + 0.006*"hear" + 0.006*"keep" + 0.005*"power" + 0.005*"anything" + 0.005*"enough" + 0.005*"end" + 0.005*"course" + 0.004*"probably" + 0.004*"money" 

Topic: 5, 
 Words: 0.015*"law" + 0.013*"government" + 0.012*"israel" + 0.012*"gun" + 0.008*"mr"

In [ ]:
###     TOPICS 
## topic 0 : Space and technology
## topic 1 : Automobile and motercycles
## topic 2 : Computer hardware and software
## topic 3 : Baseball , hockey , sports
## topic 4 : Politics ??
## topic 5 : Politics....
## topic 6 : Atheism

In [22]:
##hdp model

In [23]:
hdp = HdpModel(corpus=bow_corpus,id2word=id2word,chunksize=100)

In [24]:
len(hdp.print_topics())

20

In [27]:
for num, topic in hdp.print_topics(20):
    print('Topic: {}, \n Words: {} \n'.format(num,topic))

Topic: 0, 
 Words: 0.003*file + 0.003*god + 0.003*government + 0.003*law + 0.002*program + 0.002*gun + 0.002*information + 0.002*window + 0.002*space + 0.002*game 

Topic: 1, 
 Words: 0.008*drive + 0.007*window + 0.007*card + 0.006*scsi + 0.005*mb + 0.005*bit + 0.005*file + 0.005*disk + 0.004*mac + 0.003*driver 

Topic: 2, 
 Words: 0.012*god + 0.004*reason + 0.004*evidence + 0.004*exist + 0.003*faith + 0.003*jesus + 0.003*christian + 0.003*word + 0.003*argument + 0.003*truth 

Topic: 3, 
 Words: 0.017*db + 0.005*window + 0.005*car + 0.004*bit + 0.003*game + 0.003*chip + 0.003*card + 0.003*buy + 0.002*inc + 0.002*price 

Topic: 4, 
 Words: 0.253*max + 0.018*ax + 0.011*mw + 0.008*mq + 0.008*ml + 0.008*pl + 0.007*mv + 0.007*md + 0.007*mt + 0.007*mg 

Topic: 5, 
 Words: 0.010*card + 0.010*window + 0.006*driver + 0.006*drive + 0.004*file + 0.004*version + 0.004*scsi + 0.004*mb + 0.004*controller + 0.004*video 

Topic: 6, 
 Words: 0.006*bank + 0.006*gordon + 0.005*team + 0.004*car + 0.004*pl

In [28]:
#### coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_processed, dictionary=id2word, coherence='c_v')

In [29]:
coherence_lda = coherence_model_lda.get_coherence()

In [30]:
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5766782298654541


In [31]:
############ lsi model
lsi_model = gensim.models.lsimodel.LsiModel(corpus=bow_corpus, id2word=id2word, num_topics=7, decay=0.5)

In [32]:
for num, topic in lsi_model.print_topics(-1):
    print('Topic: {}, \n Words: {} \n'.format(num,topic))

Topic: 0, 
 Words: -0.997*"max" + -0.050*"ax" + -0.018*"pl" + -0.015*"ml" + -0.014*"mg" + -0.013*"mi" + -0.013*"tm" + -0.012*"mq" + -0.012*"gq" + -0.012*"ei" 

Topic: 1, 
 Words: -0.511*"file" + -0.206*"program" + -0.193*"image" + -0.154*"entry" + -0.132*"jpeg" + -0.128*"information" + -0.117*"output" + -0.115*"version" + -0.111*"window" + -0.103*"ftp" 

Topic: 2, 
 Words: 0.962*"db" + 0.086*"byte" + 0.082*"bh" + 0.082*"bl" + 0.079*"bit" + 0.076*"di" + -0.072*"file" + 0.065*"si" + 0.064*"al" + 0.058*"cx" 

Topic: 3, 
 Words: -0.517*"file" + -0.292*"entry" + -0.232*"output" + 0.110*"datum" + -0.107*"gun" + -0.106*"printf" + 0.103*"window" + -0.102*"char" + 0.096*"support" + -0.093*"contest" 

Topic: 4, 
 Words: -0.402*"image" + -0.346*"jpeg" + 0.180*"mr" + -0.168*"color" + -0.154*"format" + -0.149*"gif" + -0.126*"version" + 0.122*"entry" + 0.119*"gun" + 0.117*"president" 

Topic: 5, 
 Words: -0.404*"entry" + 0.303*"file" + 0.258*"gun" + 0.231*"mr" + 0.221*"jpeg" + -0.200*"output" + -0.1

In [33]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      25.038853        1       1 -0.147216 -0.076617
0      18.568789        1       2 -0.041584  0.176185
2      13.862036        1       3  0.061423  0.269415
5      13.341962        1       4 -0.161909 -0.062130
6      12.744798        1       5 -0.215716 -0.017447
3       9.588062        1       6  0.131761 -0.305179
1       6.855496        1       7  0.373241  0.015773, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
475   Default  4052.000000          god  4052.000000  30.0000  30.0000
1690  Default  1746.000000          max  1746.000000  29.0000  29.0000
4     Default  1734.000000          car  1734.000000  28.0000  28.0000
649   Default  1887.000000         team  1887.000000  27.0000  27.0000
994   Default  2230.000000       window  2230.000000  26.0000  26.0000
2178  Default  2121.000000     evidence  2121.000000  25.0000  25.0000
1148  Default  1793.000000         game  1793.000000  24.0000  24.0000
292   Default  2061.000000         file  2061.000000  23.0000  23.0000
858   Default  2009.000000          law  2009.000000  22.0000  22.0000
321   Default  2744.000000        drive  2744.000000  21.0000  21.0000
30    Default  1874.000000         card  1874.000000  20.0000  20.0000
235   Default  1777.000000   government  1777.000000  19.0000  19.0000
896   Default  2402.000000       reason  2402.000000  18.0000  18.0000
1303  Default  1613.000000       israel  1613.000000  17.0000  17.0000
1427  Default  1607.000000          gun  1607.000000  16.0000  16.0000
112   Default  1348.000000         play  1348.000000  15.0000  15.0000
643   Default  1851.000000        space  1851.000000  14.0000  14.0000
138   Default  1683.000000         chip  1683.000000  13.0000  13.0000
764   Default  2097.000000      program  2097.000000  12.0000  12.0000
799   Default  1501.000000        claim  1501.000000  11.0000  11.0000
488   Default  1177.000000        jesus  1177.000000  10.0000  10.0000
403   Default  1410.000000   technology  1410.000000   9.0000   9.0000
159   Default  1311.000000        sense  1311.000000   8.0000   8.0000
2590  Default   837.000000        video   837.000000   7.0000   7.0000
468   Default  1049.000000        faith  1049.000000   6.0000   6.0000
1388  Default   913.000000     internet   913.000000   5.0000   5.0000
99    Default  1064.000000          mac  1064.000000   4.0000   4.0000
823   Default  1316.000000       driver  1316.000000   3.0000   3.0000
148   Default  1705.000000  information  1705.000000   2.0000   2.0000
196   Default  1407.000000     software  1407.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
3567   Topic7   343.230927       seller   344.200287   2.6773  -5.2857
4627   Topic7   269.122467           ut   270.027618   2.6768  -5.5289
2847   Topic7   253.899872   motorcycle   254.797897   2.6766  -5.5871
5172   Topic7   251.675919         hawk   252.580048   2.6765  -5.5959
2031   Topic7   249.477173           mu   250.374863   2.6765  -5.6047
5333   Topic7   249.032516           mc   249.929886   2.6765  -5.6065
1474   Topic7   247.175415          nec   248.075104   2.6765  -5.6140
875    Topic7   245.055908           mo   245.953690   2.6765  -5.6226
909    Topic7   243.009125       saturn   243.909866   2.6764  -5.6310
5934   Topic7   247.806335           ww   248.728546   2.6764  -5.6114
3437   Topic7   241.235870           mk   242.137314   2.6764  -5.6383
3442   Topic7   233.725540           mv   234.623047   2.6763  -5.6699
1707   Topic7   227.786331      benefit   228.696548   2.6761  -5.6957
3432   Topic7   225.684357           lk   226.587784   2.6761  -5.7049
1700   Topic7   225.956818       sunday   226.865952   2.6761  -5.7037
4      Topic7  1723.233887          car  1734.619629   2.6735  